In [7]:
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException  
from progress.bar import IncrementalBar
import csv
import time



driver = webdriver.Chrome()  # Point the path to where you've installed chromedriver

driver.get("https://www.linkedin.com/login")





In [8]:
driver.get("https://www.linkedin.com/mynetwork/invite-connect/connections")


In [9]:
from selenium.common.exceptions import  TimeoutException  

links = []

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)  # Wait for the new connections to load

    # Fetch the URLs of each connection
    cards = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.mn-connection-card__link')))
    
    new_links = [card.get_attribute('href') for card in cards]
    
    # If no new connections were loaded, break the loop
    if set(new_links) == set(links):
        break
    
    links = new_links
    


In [10]:
len(links)

79

In [11]:
with open("scrape_email.csv","w", newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = ['E-mail'])
    writer.writeheader()
    
    bar = IncrementalBar('Fetching Connections', max=len(links))
    
    for link in links:
        driver.get(link)
        
        # Click on 'Contact info' link on the connection's page
        try:
            contact_info_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, 'Contact info')))
            contact_info_button.click()
            
            # Scrape the email from the contact info page
            # Scrape the email from the contact info page
            email_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pv-contact-info__ci-container.t-14 a[href^='mailto:']")))
            email = email_element.get_attribute("href").replace("mailto:", "")
        except TimeoutException:
            email = ""
        
        writer.writerow({'E-mail': email})
        bar.next()
        
    bar.finish()

KeyboardInterrupt: 